In [40]:
import numpy as np

In [3]:
!ls ./cifar-10-batches-py


batches.meta data_batch_2 data_batch_4 readme.html
data_batch_1 data_batch_3 data_batch_5 test_batch


In [22]:
file = './cifar-10-batches-py/test_batch'
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo,encoding='bytes')
    return dict

dd = unpickle(file)

In [30]:
dd.keys()

dict_keys([b'labels', b'data', b'filenames', b'batch_label'])

In [77]:
raw = dd[b'data']
ydata = dd[b'labels']

## 损失函数

In [70]:
def L(X, y, W):
    """
    fully-vectorized implementation :
    - X holds all the training examples as columns (e.g. 3073 x 50,000 in CIFAR-10)
    - y is array of integers specifying correct class (e.g. 50,000-D array)
    - W are weights (e.g. 10 x 3073)
    """
    delta = 1.0
    scores = W.dot(X.T)
    lidx = range(0, len(y))
    margins = np.maximum(0, scores - scores[y, lidx] + delta)
    margins[y, lidx] = 0
    loss = np.sum(margins)/(len(y))
    return loss

In [107]:

xdata =[]
for x in raw:
    xdata.append(np.insert(x,0,1))
xdata = np.array(xdata)

1 loop, best of 3: 164 ms per loop


In [108]:

# 每个样本增加一个bias 1

# 法一
# xdata =[]
# for x in raw:
#     xdata.append(np.insert(x,0,1))
# xdata = np.array(xdata)

#法二
xdata = np.column_stack((raw, np.ones(len(raw))))

# 法三
# xdata = np.ones((raw.shape[0],raw.shape[1]+1))
# xdata[:,:-1] = raw

In [109]:
xdata.shape,raw.shape

((10000, 3073), (10000, 3072))

In [113]:
W = np.random.randn(10, 3073) * 0.0001

L(xdata,ydata,W)

10.081358115448854

## 优化

In [114]:
X_train = xdata
Y_train = ydata

In [118]:
# 随机搜索
bestloss = float("inf") # Python assigns the highest possible float value
for num in range(100):
    W = np.random.randn(10, 3073) * 0.0001 # generate random parameters
    loss = L(X_train, Y_train, W) # get the loss over the entire training set
    if loss < bestloss: # keep track of the best solution
        bestloss = loss
        bestW = W
print('in attempt %d the loss was %f, best %f' % (num, loss, bestloss))

in attempt 99 the loss was 8.983980, best 8.983980


In [131]:
#随机下降法
W = np.random.randn(10, 3073) * 0.001 # generate random starting W
bestloss = float("inf")
for i in range(1000):
    step_size = 0.00001
    Wtry = W + np.random.randn(10, 3073) * step_size
    loss = L(X_train, Y_train, Wtry)
    if loss < bestloss:
        W = Wtry
        bestloss = loss
    print('iter %d loss is %f' % (i, bestloss))

iter 0 loss is 41.161338
iter 1 loss is 41.161338
iter 2 loss is 41.161338
iter 3 loss is 41.104320
iter 4 loss is 41.075899
iter 5 loss is 41.075899
iter 6 loss is 40.861117
iter 7 loss is 40.679085
iter 8 loss is 40.640761
iter 9 loss is 40.521532
iter 10 loss is 40.521532
iter 11 loss is 40.521532
iter 12 loss is 40.442019
iter 13 loss is 40.442019
iter 14 loss is 40.442019
iter 15 loss is 40.216925
iter 16 loss is 40.216925
iter 17 loss is 40.104837
iter 18 loss is 40.104837
iter 19 loss is 40.104837
iter 20 loss is 40.104837
iter 21 loss is 40.000980
iter 22 loss is 40.000980
iter 23 loss is 39.901984
iter 24 loss is 39.901984
iter 25 loss is 39.783670
iter 26 loss is 39.783670
iter 27 loss is 39.577417
iter 28 loss is 39.564077
iter 29 loss is 39.546214
iter 30 loss is 39.546214
iter 31 loss is 39.546214
iter 32 loss is 39.392918
iter 33 loss is 39.392918
iter 34 loss is 39.360285
iter 35 loss is 39.137723
iter 36 loss is 39.122535
iter 37 loss is 39.079045
iter 38 loss is 39.002

In [149]:
# 梯度下降法
def eval_numerical_gradient(f, x):
    """ 
    a naive implementation of numerical gradient of f at x 
    - f should be a function that takes a single argument
    - x is the point (numpy array) to evaluate the gradient at
    """ 

    fx = f(x) # evaluate function value at original point
    grad = np.zeros(x.shape)
    h = 0.00001

    # iterate over all indexes in x
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:

        # evaluate function at x+h
        ix = it.multi_index
        old_value = x[ix]
        x[ix] = old_value + h # increment by h
        fxh = f(x) # evalute f(x + h)
        x[ix] = old_value # restore to previous value (very important!)

        # compute the partial derivative
        grad[ix] = (fxh - fx) / h # the slope
        it.iternext() # step to next dimension

    return grad

# 10000个太多了,测试1000个
k=100
def CIFAR10_loss_fun(W):
    return L(X_train[:k], Y_train[:k], W)

In [150]:
%%time
W = np.random.rand(10, 3073) * 0.001 # random weight vector

df = eval_numerical_gradient(CIFAR10_loss_fun, W) # get the gradient

CPU times: user 18.6 s, sys: 216 ms, total: 18.8 s
Wall time: 19.3 s


In [153]:

%%time
loss_original = CIFAR10_loss_fun(W) # the original loss
print('original loss: %f' % (loss_original, ))

for step_size_log in [-10, -9, -8, -7, -6, -5,-4,-3,-2,-1]:
    step_size = 10 ** step_size_log
    W_new = W - step_size * df # new position in the weight space
    loss_new = CIFAR10_loss_fun(W_new)
    print('for step size %f new loss: %f' % (step_size, loss_new))

original loss: 18.754741
for step size 0.000000 new loss: 18.740895
for step size 0.000000 new loss: 18.616471
for step size 0.000000 new loss: 17.402123
for step size 0.000000 new loss: 10.438134
for step size 0.000001 new loss: 124.150568
for step size 0.000010 new loss: 1328.548602
for step size 0.000100 new loss: 13372.797059
for step size 0.001000 new loss: 133815.281625
for step size 0.010000 new loss: 1338240.127288
for step size 0.100000 new loss: 13382488.583912
CPU times: user 15.1 ms, sys: 2.49 ms, total: 17.6 ms
Wall time: 15.2 ms


**结论:**要选择大小合适的stepsize

- 梯度方向其实是函数增加最快的方向
- 梯度下降法是沿着负梯度方向